In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras_tuner as kt
from sklearn.preprocessing import MinMaxScaler
import os

In [ ]:
#loading the trajectories with two peaks and their corresponding labels
xtrain2_1 = np.loadtxt('../Data/Xtrainx2_1.csv', delimiter=',')
xtrain2_2 = np.loadtxt('../Data/Xtrainx2_2.csv', delimiter=',')
#files with trajectories were split to be small enough to upload to GitHub, here we concatenate them
xtrain2 = np.concatenate((xtrain2_1, xtrain2_2))
ytrain2 = np.loadtxt('../Data/Ytrain2.csv', delimiter=',')

In [ ]:
#ytrain3[:,[2,6]] selects the columns with the positions of the three peaks, located at indices 2, and 6. 
ytrain = ytrain2[:,[2,6]]

#sorting the positions of the peaks for consistency
for i in range(ytrain.shape[0]):
    ytrain[i,:] = ytrain[i,ytrain[i,:].argsort()]

In [ ]:
#scale the labels using MinMaxScaler for normalisation
scaler = MinMaxScaler()
ytrain_scaled = scaler.fit_transform(ytrain)

In [ ]:
#function to split the data into training, validation and test sets, and calculate their Fourier coefficients along
#with the corresponding labels.
def fouriertrainvaltest(X, Y, Ntrain, Nval, Ntest):

    #Generating a training set with Ntrain trajectories, a validation with Nval trajectories and a test set with
    #Ntest trajectories. The original trajectories contain 800 time steps but we only use 400 of them, we thus take
    #every second point
    Xtrain = X[0:Ntrain, 0:800:2]
    Xval = X[Ntrain:Ntrain+Nval, 0:800:2]
    Xtest = X[Ntrain+Nval:Ntrain+Nval+Ntest, 0:800:2]

    #extract the corresponding labels for the training, validation and test sets.
    Ytrain = Y[0:Ntrain, :]
    Yval = Y[Ntrain:Ntrain+Nval, :]
    Ytest = Y[Ntrain+Nval:Ntrain+Nval+Ntest, :]
    
    #calculating the Fourier coefficients for each subset.
    XtrainF = np.fft.fft(Xtrain)
    XvalF = np.fft.fft(Xval)
    XtestF = np.fft.fft(Xtest)

    #Prepare to split the Fourier coefficients into their real and imaginary components. Each complex number will 
    #occupy two columns: one for the real part and one for the imaginary part. Therefore, we create new arrays that 
    #have twice the number of columns. 
    xtrain = np.zeros((XtrainF.shape[0], 2*XtrainF.shape[1]))
    xval = np.zeros((XvalF.shape[0], 2*XvalF.shape[1]))
    xtest = np.zeros((XtestF.shape[0], 2*XtestF.shape[1]))

    #For each Fourier coefficient in the training set, split into real and imaginary parts. These parts are then
    #stored alternately (even indices for real, odd indices for imaginary).
    for i in range(XtrainF.shape[0]):
        for j in range(XtrainF.shape[1]):
            xtrain[i, 2*j] = XtrainF[i,j].real
            xtrain[i, 2*j + 1] = XtrainF[i,j].imag

    #Do the same for the test set, splitting the Fourier coefficients into their real and imaginary parts.
    for i in range(XtestF.shape[0]):
        for j in range(XtestF.shape[1]):
            xtest[i, 2*j] = XtestF[i,j].real
            xtest[i, 2*j + 1] = XtestF[i,j].imag

    #Similarly, split the Fourier coefficients for the validation set.
    for i in range(XvalF.shape[0]):
        for j in range(XvalF.shape[1]):
            xval[i, 2*j] = XvalF[i,j].real
            xval[i, 2*j + 1] = XvalF[i,j].imag

    #Return the transformed training, validation and test sets along with their corresponding labels
    return(xtrain, xval, xtest, Ytrain, Yval, Ytest)

In [ ]:
#Function to calculate the R-squared metric. This function takes the true and predicted values, and calculates the 
#R-squared value
def r_square(y_true, y_pred):
    ss_res = tf.reduce_sum(tf.square(y_true - y_pred))
    ss_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    return(1 - ss_res/ss_tot)

In [ ]:
#generating a training, validation and test set so the dimensions can be used to load the hyperparameters
xtrainf, xval, xtest, ytrainf, yval, ytest = fouriertrainvaltest(xtrain2, ytrain_scaled, 4800, 600, 600)

In [ ]:
#define a HyperModel class for the Keras Tuner to optimise the architecture and hyperparameters of the model.
class HyperModel(kt.HyperModel):

    #function to build the model with hyperparmaeter tuning
    def build(self, hp):
        #create a sequential model
        model = tf.keras.Sequential()

        #Tune the number of neurons in the first dense layer between 32 and 512, with a step of 32. 'units_0' is the 
        #hyperparameter name, and it will be varied during tuning.
        model.add(tf.keras.layers.Dense(
            units=hp.Int('units_0', min_value = 32, max_value = 512, step=32),
            input_dim = (xtrainf.shape[1]),
            activation='relu'))

        #Tune the number of additional hidden layers between 0 and 10. For each layer, tune the number of neurons
        #between 32 and 512.
        for i in range(hp.Int('layers', 0, 10)):
            model.add(tf.keras.layers.Dense(
                units=hp.Int('units_' + str(i + 1), min_value=32, max_value=512, step=32),
                activation='relu'))
        
        #Add the output layer with 2 neurons (corresponding to the two peak positions in the regression task). Use 
        #the linear activation function for regression outputs
        model.add(tf.keras.layers.Dense(2,
                activation='linear'))


        #Tune the learning rate of the Adam optimiser, choosing from [0.01, 0,001, 0.0001, 0.00001, 0.000001]
        hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5, 1e-6])
        
        #Compile the model with the Adam optimiser, mean squared error loss, and r-squared metric
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = hp_learning_rate),
                      loss="mean_squared_error",
                      metrics=[r_square])
        
        #return the constructed model
        return(model)

    #function to fit the model, allowing the batch size to be tuned as well
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            #Tune the batch size by selecting from [16, 32, 61, half of the training set, or the full training set]
            batch_size=hp.Choice("batch_size", [16, 32, 64, int(xtrainf.shape[0]/2), xtrainf.shape[0]]),
            **kwargs,
        )
    
#instantiate a bayesian optimisation tuner
tuner = kt.BayesianOptimization(HyperModel(), #pass the HyperModel class
                     objective='val_loss', #Objective is to minimise the validation loss
                     max_trials = 100, #Perform up to 100 trials to explore different hyperparameter combinations
                     project_name='hp_optimisation_RCregression_twoRCsnu') #project name 

In [ ]:
#Retrieve the best hyperparameters from the search process
best_hps=tuner.get_best_hyperparameters(num_trials=100)[0] #Get the top hyperparameter combination from 100 trials

#print statements to display the optimal hyperparameters
print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units_0')}""")

#print the optimal number of hidden layers
print(f""" The optimal number of hidden layers is {best_hps.get('layers')}""")

#loop through and print the optimal number of units for each hidden layer
for i in range(best_hps.get('layers')):
    print(f""" The optimal number of units in layer {i + 1} is {best_hps.get('units_' + str(i + 1))}""")

#print the optimal learning rate and batch size
print(f"""the optimal learning rate for the optimizer is {best_hps.get('learning_rate')} and the optimal batch size is {best_hps.get('batch_size')}""")

In [ ]:
#instantiate a new model using the HyperModel class with the best hyperparameters obtained from tuning
model = HyperModel().build(best_hps)

#define the path where the model's weights will be saved
checkpoint_path = "training_RCregression_twopeaksnu.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)

#Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
#fit the model on the training data with validation
history = model.fit(xtrainf, ytrainf, epochs = 1000, validation_data = (xval, yval), batch_size = best_hps.get('batch_size'), verbose=1, callbacks=[cp_callback])

In [ ]:
#plotting the training and validation loss values over epochs
plt.plot(history.history['loss'], label='training')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('loss')
plt.savefig('lossvepochs_RCregression_twopeaksnu.pdf')

In [ ]:
#plotting the training and validation r-squared value over epochs
plt.plot(history.history['r_square'], label='training')
plt.plot(history.history['val_r_square'], label='validation')
plt.legend()
plt.xlabel('epochs')
plt.ylabel(r"$R^2$")
plt.savefig('r2vepochs_RCregression_twopeaksnu.pdf')

In [ ]:
#evaluating the model on the test set
model.evaluate(xtest, ytest)

In [ ]:
#evaluating the model on the validation set
model.evaluate(xval, yval)

In [ ]:
#evaluating the model on the training set 
model.evaluate(xtrainf, ytrainf)

In [ ]:
#inverse transform the scaled outputs to get original values
y_true = scaler.inverse_transform(ytest)
y_pred = scaler.inverse_transform(model.predict(xtest))

In [ ]:
#configure font properties for the plots
plt.rc('font',family='Times New Roman')
plt.rcParams.update({'font.size':13})
plt.rcParams['font.size'] = 13
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.rm'] = 'Times New Roman'
plt.rcParams['axes.linewidth'] = 1

In [ ]:
#Create a scatter plot of true values for $|nu_1$ versus predicted values
plt.scatter(y_true[:,0], y_pred[:,0], marker = 'o', color='blue', label='Predictions')
min_val = min(min(y_true[:,0]), min(y_pred[:,0]))
max_val = max(max(y_true[:,0]), max(y_pred[:,0]))
plt.plot([min_val, max_val], [min_val, max_val], label='True', color='orange', linewidth = 2.5)
plt.xlabel(r"$\mathrm{\nu_1 / \omega_0}$")
plt.ylabel(r"$\mathrm{\hat{\nu}_1 / \omega_0}$")
plt.legend()
plt.savefig('twopeakspredictednu1vreal.pdf', bbox_inches='tight')
plt.show()

In [ ]:
#Create a scatter plot of true values for $|nu_2$ versus predicted values
plt.scatter(y_true[:,1], y_pred[:,1], marker = 'o', color='blue', label='Predictions')
min_val = min(min(y_true[:,1]), min(y_pred[:,1]))
max_val = max(max(y_true[:,1]), max(y_pred[:,1]))
plt.plot([min_val, max_val], [min_val, max_val], label='True', color='orange', linewidth = 2.5)
plt.xlabel(r"$\mathrm{\nu_2 / \omega_0}$")
plt.ylabel(r"$\mathrm{\hat{\nu}_2 / \omega_0}$")
plt.legend()
plt.savefig('twopeakspredictednu2vreal.pdf', bbox_inches='tight')
plt.show()

In [ ]:
#define an array where each element is the difference between the predicted value and true value for $|nu_1$ for a 
#given trajectory
diffs1 = y_pred[:,0] - y_true[:,0]
#print the minimum difference to find the smallest prediction error
print(np.amin(diffs1))
#print the maximum difference to find the largest prediction error
print(np.amax(diffs1))

In [ ]:
#define intervals for analysing the differences between predicted and true values for $|nu_1$
intervals1 = [-0.06, -0.0575, -0.055, -0.0525, -0.05, -0.0475, -0.045, -0.0425, -0.04, -0.0375, -0.035, -0.0325, -0.03, -0.0275, -0.025, -0.0225, -0.02, -0.0175, -0.015, -0.0125, -0.01, -0.0075, -0.005, -0.0025, 0, 0.0025, 0.005, 0.0075, 0.01, 0.0125, 0.015, 0.0175, 0.02, 0.0225, 0.025, 0.0275, 0.03, 0.0325, 0.035, 0.0375, 0.04, 0.0425, 0.045, 0.0475, 0.05, 0.0525, 0.055, 0.0575, 0.06, 0.0625, 0.065, 0.0675, 0.07, 0.0725, 0.075, 0.0775, 0.08]

In [ ]:
#initialise list to store the frequencies of differences in intervals for $|nu_1$
freq1 = []

#loop through each interval
for i in range(len(intervals1)-1):
    #create a mask to find differences within the current interval
    mask1 = (diffs1 >= intervals1[i]) & (diffs1 < intervals1[i+1])
    #append the count of differences within the current interval to the frequency list
    freq1.append(len(diffs1[mask1]))

In [ ]:
#calculate x positions for bars and heights
x1 = intervals1[1:] 
heights1 = (np.array(freq1)/600)*100

#create the bar chart
plt.bar(x1, heights1, width=-0.0025, align='edge', alpha = 1, color = 'orange', edgecolor='black')  
plt.xlabel(r"$\mathrm{(\hat{\nu}_1 - \nu_1)/\omega_0}$")
plt.ylabel('% of trajectories')
plt.savefig('twopeaksbarchartnu1.pdf', bbox_inches='tight')
plt.show()

In [ ]:
#define an array where each element is the difference between the predicted value and true value for $|nu_2$ for a 
#given trajectory
diffs2 = y_pred[:,1] - y_true[:,1]

#print the minimum difference to find the smallest prediction error
print(np.amin(diffs2))

#print the maximum difference to find the largest prediction error
print(np.amax(diffs2))

In [ ]:
#define intervals for analysing the differences between predicted and true values for $|nu_2$
intervals2 = [-0.09, -0.0875, -0.085, -0.0825, -0.08, -0.0775, -0.075, -0.0725, -0.07, -0.0675, -0.065, -0.0625, -0.06, -0.0575, -0.055, -0.0525, -0.05, -0.0475, -0.045, -0.0425, -0.04, -0.0375, -0.035, -0.0325, -0.03, -0.0275, -0.025, -0.0225, -0.02, -0.0175, -0.015, -0.0125, -0.01, -0.0075, -0.005, -0.0025, 0, 0.0025, 0.005, 0.0075, 0.01, 0.0125, 0.015, 0.0175, 0.02, 0.0225, 0.025, 0.0275, 0.03, 0.0325, 0.035, 0.0375, 0.04, 0.0425, 0.045, 0.0475, 0.05, 0.0525, 0.055, 0.0575, 0.06, 0.0625, 0.065, 0.0675, 0.07, 0.0725]

In [ ]:
#initialise list to store the frequencies of differences in intervals for $|nu_2$
freq2 = []

#loop through each interval
for i in range(len(intervals2)-1):
    #create a mask to find differences within the current interval
    mask2 = (diffs2 >= intervals2[i]) & (diffs2 < intervals2[i+1])
    #append the count of differences within the current interval to the frequency list
    freq2.append(len(diffs2[mask2]))

In [ ]:
#calculate x positions for bars and heights
x2 = intervals2[1:] 
heights2 = (np.array(freq2)/600)*100

#create the bar chart
plt.bar(x2, heights2, width=-0.0025, align='edge', alpha = 1, color = 'orange', edgecolor='black') 
plt.xlabel(r"$\mathrm{(\hat{\nu}_2 - \nu_2)/\omega_0}$")
plt.ylabel('% of trajectories')
plt.savefig('twopeaksbarchartnu2.pdf', bbox_inches='tight')
plt.show()